In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
#importing data
raw_data = pd.read_csv(r"C:\Users\USER\Crime_Data_from_2020_to_Present.csv")

data = raw_data.copy()

#Renaming all columns(on the copy)
data.columns = ['File_Number','Report_Date','Occ_Date','Occ_Time','Area_Code','Area_Name',
                'Sub_Area_Code','Crime_degree','Crime_Code','Crime_Desc','Mocodes','Vict_Age',
                'Vict_Sex','Vict Descent','Place_code','Place_Desc','Weapon_code','Weapon_Desc',
                'Status','Status_Desc','crime_1','crime_2','crime_3','crime_4','Location',
                'Cross_Street','LAT','LON'
               ]

In [16]:
data.head(20).to_clipboard()